#### Installing Necessary Libraries

In [ ]:
!pip3 install scikit-learn
!pip3 install bokeh
!pip3 install torchvision==0.9.0
!pip3 install scikit-learn
!pip3 install torch==1.8.0
!pip3 install onnx==1.8.1
!pip3 install onnxruntime==1.7.0
!pip3 install vizer
!pip3 install yacs

#### Importing Libraries

In [ ]:
import cv2
import glob
import os
import numpy as np
from PIL import Image
import re
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import cv2
import glob
import os
import numpy as np
from PIL import Image
import re
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import torch
import sklearn
from bokeh import plotting
from numpy.linalg import norm
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
import math
from IPython.display import Image, display

## Getting the dataset


In [ ]:
# User needs to download the dataset of their choice. 

## Getting The Model

### Clonning the repo and getting the onnx model

In [ ]:
%%bash
git clone https://github.com/lufficc/SSD.git
cd SSD/
git reset --hard 68dc0a20efaf3997e58b616afaaaa21bf8ca3c05
wget https://github.com/lufficc/SSD/releases/download/1.2/mobilenet_v2_ssd320_voc0712_v2.pth

In [ ]:
command1="cp utils/changes_on_top_without_ABP-NMS.patch SSD/;cd SSD;patch -p1 < ./changes_on_top_without_ABP-NMS.patch;python demo.py --config-file configs/mobilenet_v2_ssd320_voc0712.yaml --images_dir demo  --ckpt mobilenet_v2_ssd320_voc0712_v2.pth"
os.system(command1)


In [ ]:
command2="cd SSD;cp ssd_mobilenetV2_without_ABP-NMS.onnx ../"
os.system(command2)

### Getting the DLC Model

In [ ]:
%%bash
snpe-onnx-to-dlc -i ssd_mobilenetV2_without_ABP-NMS.onnx -o ssd_mobilenetV2_without_ABP-NMS_fp32.dlc --quantization_overrides encoding_format.encodings 

#### Architecture of The SSD-MobileNet-V2 Model
*Please Check From the Below File Path the Architecture*

In [ ]:
%%bash
snpe-dlc-viewer -i ssd_mobilenetV2_without_ABP-NMS_fp32.dlc -s ssd_mobilenetV2_without_ABP-NMS_fp32.html

### Preprocessing the image

In [ ]:
class Compose(object):
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, img, boxes=None, labels=None):
        for t in self.transforms:
            img, boxes, labels = t(img, boxes, labels)
            if boxes is not None:
                boxes, labels = remove_empty_boxes(boxes, labels)
        return img, boxes, labels

class SubtractMeans(object):
    def __init__(self, mean):
        self.mean = np.array(mean, dtype=np.float32)

    def __call__(self, image, boxes=None, labels=None):
        image = image.astype(np.float32)
        image -= self.mean
        return image.astype(np.float32), boxes, labels

class Resize(object):
    def __init__(self, size=300):
        self.size = size
    def __call__(self, image, boxes=None, labels=None):
        image = cv2.resize(image, (self.size,
                                   self.size))
        return image, boxes, labels

def build_transforms(IMAGE_SIZE = 300, PIXEL_MEAN = [123, 117, 104]):
    transform = [
        Resize(IMAGE_SIZE), #300
        SubtractMeans(PIXEL_MEAN), #[123, 117, 104]
    ]
    transform = Compose(transform)
    return transform

def load_img(img):
    image_bgr = cv2.imread(img, cv2.IMREAD_COLOR)
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    height, width, _ = image_rgb.shape
    transforms = build_transforms(IMAGE_SIZE = 320, PIXEL_MEAN = [123, 117, 104])
    images = transforms(image_rgb)[0]
    images = np.ascontiguousarray(images)
    return images

In [ ]:
import cv2
import numpy as np
import os
def detect(imgfile,i):
    img= load_img(imgfile)
    img.tofile("raw/"+filenames[i].split(".")[0]+".raw")
    
filenames = os.listdir("val2017")
for i in range(0,len(filenames)):
    if "jpg" in filenames[i].lower():
        detect("val2017/"+filenames[i],i)
    else:
        print("Could not get this image pre-processed")
        print(" ==> filename :: ",filenames[i].lower())

In [ ]:
%%bash
find ./raw -name *.raw > list.txt

#### Quantizing the DLC Model

In [ ]:
%%bash
snpe-dlc-quantize --input_dlc ssd_mobilenetV2_without_ABP-NMS_fp32.dlc  --override_params --optimization cle --input_list list.txt --use_enhanced_quantizer --use_adjusted_weights_quantizer --axis_quant --output_dlc ssd_mobilenetV2_without_ABP-NMS_a8w8.dlc --enable_htp --htp_socs sm8550

## Running The DLC Model

In [ ]:
%%bash
cd ../../../../snpe/qaisw-v2.15.1.230926150623_62883
adb devices

In [ ]:
## Give appropriate permission to the directory "FOLDER_WITH_ARTIFACTS" you are working with
import os
os.environ['SNPE_ROOT']="../../../../snpe/qaisw-v2.15.1.230926150623_62883"#set up your snpe path here.
os.environ['RAW_FILE_FOLDER']="raw"#rawfiles
os.environ['FOLDER_WITH_ARTIFACTS']="SSD_MobileNetV2"
os.environ['DLC32']="ssd_mobilenetV2_without_ABP-NMS_fp32.dlc"
os.environ['DLC8']="ssd_mobilenetV2_without_ABP-NMS_a8w8.dlc"
os.environ['TARGET_INPUT_LIST']="list.txt"
os.environ['ONDEVICE_FOLDER']="SSD_MobileNetV2"
os.environ['DEVICE_HOST']="localhost"
os.environ['DEVICE_ID']="389c94f8" #fill your device-id. Use command "adb devices" to get devices names. example :"e18d5d0"
os.environ['SNPE_TARGET_ARCH']="aarch64-android"
os.environ['SNPE_TARGET_STL']="libc++_shared.so"

#### Creating Bin and Lib Folder on Device

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell "mkdir -p /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/bin" && $DEVICE_SHELL shell "mkdir -p /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/lib" && $DEVICE_SHELL shell "mkdir -p /data/local/tmp/snpeexample/dsp/lib"

### Pushing all Lib and Bin files onto Device

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL push $SNPE_ROOT/lib/$SNPE_TARGET_ARCH/$SNPE_TARGET_STL /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/lib
$DEVICE_SHELL push $SNPE_ROOT/bin/$SNPE_TARGET_ARCH/snpe-net-run /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/bin
$DEVICE_SHELL push $SNPE_ROOT/lib/hexagon-v73/unsigned/*.so /data/local/tmp/snpeexample/dsp/lib
$DEVICE_SHELL push $SNPE_ROOT/lib/$SNPE_TARGET_ARCH/*.so /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/lib

### Pushing Artifacts on to Device

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell "mkdir -p /data/local/tmp/$ONDEVICE_FOLDER"

In [ ]:
%%bash

export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL push $DLC32 /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push $DLC8 /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push raw /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push $TARGET_INPUT_LIST /data/local/tmp/$ONDEVICE_FOLDER

### Inferencing 8 bit DLC on DSP Runtime


In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/data/local/tmp/snpeexample/aarch64-android/lib
export PATH=$PATH:/data/local/tmp/snpeexample/aarch64-android/bin
export OUTPUT_FOLDER=OUTPUT_8b_DSP
export OUTPUT_DLC_QUANTIZED8=ssd_mobilenetV2_without_ABP-NMS_a8w8.dlc
export ADSP_LIBRARY_PATH="/data/local/tmp/snpeexample/dsp/lib;/system/lib/rfsa/adsp;/system/vendor/lib/rfsa/adsp;/dsp"
export ONDEVICE_FOLDER="SSD_MobileNetV2"
cd /data/local/tmp/$ONDEVICE_FOLDER &&
snpe-net-run --container $OUTPUT_DLC_QUANTIZED8 --input_list list.txt --set_output_tensors 935,986 --output_dir $OUTPUT_FOLDER --use_dsp

### Inferencing 32b DLC on CPU Runtime


In [ ]:
%%bash

export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell

export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/data/local/tmp/snpeexample/aarch64-android/lib
export PATH=$PATH:/data/local/tmp/snpeexample/aarch64-android/bin
export OUTPUT_FOLDER=OUTPUT_32b_CPU
export OUTPUT_DLC_32=ssd_mobilenetV2_without_ABP-NMS_fp32.dlc
export ONDEVICE_FOLDER="SSD_MobileNetV2"
cd /data/local/tmp/$ONDEVICE_FOLDER &&
snpe-net-run --container $OUTPUT_DLC_32 --input_list list.txt --set_unconsumed_as_output --output_dir $OUTPUT_FOLDER

### Pulling the Output

In [ ]:
%%bash
rm -rf OUTPUT_8b_DSP/
rm -rf OUTPUT_32b_CPU/

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL pull /data/local/tmp/$ONDEVICE_FOLDER/OUTPUT_8b_DSP OUTPUT_8b_DSP
$DEVICE_SHELL pull /data/local/tmp/$ONDEVICE_FOLDER/OUTPUT_32b_CPU OUTPUT_32b_CPU

## Post Processing the Output

In [ ]:
%%bash
mkdir -p "detection_results_32b_CPU"
mkdir -p "detection_results_8b_DSP"
mkdir -p "prediction_results_32b_CPU"
mkdir -p "prediction_results_8b_DSP"

In [ ]:
class_name = ["BACKGROUND",
            "aeroplane",
            "bicycle",
            "bird",
            "boat",
            "bottle",
            "bus",
            "car",
            "cat",
            "chair",
            "cow",
            "diningtable",
            "dog",
            "horse",
            "motorbike",
            "person",
            "pottedplant",
            "sheep",
            "sofa",
            "train",
            "tvmonitor"]

label2class={str(i):x for i,x in enumerate(class_name)}
print(label2class)

In [ ]:
import torch

#Computing the areas of rectangle given two corners
def area_of(left_top, right_bottom) -> torch.Tensor:
    hw = torch.clamp(right_bottom - left_top, min=0.0)
    return hw[..., 0] * hw[..., 1]

#Returning intersection-over-union (Jaccard index) of boxes.
def iou_of(boxes0, boxes1, eps=1e-5):
    overlap_left_top = torch.max(boxes0[..., :2], boxes1[..., :2])
    overlap_right_bottom = torch.min(boxes0[..., 2:], boxes1[..., 2:])

    overlap_area = area_of(overlap_left_top, overlap_right_bottom)
    area0 = area_of(boxes0[..., :2], boxes0[..., 2:])
    area1 = area_of(boxes1[..., :2], boxes1[..., 2:])
    return overlap_area / (area0 + area1 - overlap_area + eps)



#Returning the Bounding Boxes
def hard_nms(box_scores, iou_threshold, top_k=-1, candidate_size=200):
    scores = box_scores[:, -1]
    boxes = box_scores[:, :-1]
    picked = []
    _, indexes = scores.sort(descending=True)
    indexes = indexes[:candidate_size]
    while len(indexes) > 0:
        current = indexes[0]
        picked.append(current.item())
        if 0 < top_k == len(picked) or len(indexes) == 1:
            break
        current_box = boxes[current, :]
        indexes = indexes[1:]
        rest_boxes = boxes[indexes, :]
        iou = iou_of(
            rest_boxes,
            current_box.unsqueeze(0),
        )
        indexes = indexes[iou <= iou_threshold]

    return box_scores[picked, :]



def nms(box_scores, nms_method=None, score_threshold=None, iou_threshold=None,
        sigma=0.5, top_k=-1, candidate_size=200):
    if nms_method == "soft":
        return soft_nms(box_scores, score_threshold, sigma, top_k)
    else:
        return hard_nms(box_scores, iou_threshold, top_k, candidate_size=candidate_size)


In [ ]:
colors = np.random.uniform(0, 255, size=(len(list(label2class.values())), 3))

import cv2.dnn
import torch
def postProcessing(scores,boxes,original_image_path,save_image_path):
    original_image=cv2.imread(original_image_path, cv2.IMREAD_COLOR)
    height,width,_=original_image.shape
    prob_threshold = 0.4
    picked_box_probs = []
    picked_labels = []
    for class_index in range(1, scores.shape[1]):
        
        probs = scores[:, class_index]
        
        mask = probs > prob_threshold
        probs = probs[mask]
        
        if probs.shape[0] == 0:
            continue
        subset_boxes = boxes[mask, :]
        subset_boxes = torch.from_numpy(subset_boxes)
        probs = torch.from_numpy(probs)
        box_probs = torch.cat([subset_boxes, probs.reshape(-1, 1)], dim=1)
        box_probs = nms(box_probs, None,
                                  score_threshold=prob_threshold,
                                  iou_threshold=0.2,
                                  sigma=0.2,
                                  top_k=-1,
                                  candidate_size=200)
        picked_box_probs.extend([box_probs])
        picked_labels.extend([class_index] * box_probs.size(0))
        picked_box_probs = torch.cat(picked_box_probs)
        picked_box_probs[:, 0] *= width
        picked_box_probs[:, 1] *= height
        picked_box_probs[:, 2] *= width
        picked_box_probs[:, 3] *= height
        label = class_name[picked_labels[0]]
        
        for i in range(0,len(picked_box_probs)):
            x,y=int(picked_box_probs[i, 0].numpy()),int(picked_box_probs[i, 1].numpy())
            x_plus_w,y_plus_h=int(picked_box_probs[i, 2].numpy()),int(picked_box_probs[i, 3].numpy())
            original_image = cv2.rectangle(original_image,(x, y), (x_plus_w,y_plus_h),colors[class_index],2)
            original_image=cv2.putText(original_image, label,(int(picked_box_probs[i, 0])+9, int(picked_box_probs[i, 1])-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255, 40, 255),2)  # line type
        
        
        picked_box_probs = []
        picked_labels = []
    cv2.imwrite(save_image_path,original_image)

In [ ]:
folder = ["OUTPUT_32b_CPU","OUTPUT_8b_DSP"]
detection_results = ["detection_results_32b_CPU","detection_results_8b_DSP"]
results = ["prediction_results_32b_CPU","prediction_results_8b_DSP"]
f=open("list.txt","r")
input_list = f.readlines()
f.close()
for j in range(0,2):
    for result_path in glob.glob(os.path.join(folder[j], '*')):
        if ".log" not in result_path :
            #using raw file to traverse output folders and result folder will be in sequence as per list.txt
            raw_path = input_list[int(result_path.split("_")[-1])].replace("\n","") # rawfile path from list.txt
            if os.path.exists(raw_path):
                scores = np.fromfile(result_path+'/935.raw', dtype="float32")
                boxes=np.fromfile(result_path+'/986.raw', dtype="float32")

                boxes=boxes.reshape((3234,4))
                scores=scores.reshape((3234,21))
                originalImagePath = raw_path.replace(".raw",".jpg").replace("raw","val2017")

                runtime = folder[j].split("_")[-1]
                save_img_path = detection_results[j]+"/"+raw_path.split("/")[-1].split(".")[0]+"_"+runtime+".jpg"

                postProcessing(scores,boxes,originalImagePath,save_img_path)
                

## Visualization

In [ ]:

detection_results = ["detection_results_32b_CPU","detection_results_8b_DSP"]

for CPU_path in glob.glob(os.path.join(detection_results[0], '*')):
    CPU_img_name=CPU_path.split("/")[1].split("_")[0]
    for DSP_path in glob.glob(os.path.join(detection_results[1], '*')):
        DSP_img_name=DSP_path.split("/")[1].split("_")[0]
        
        if CPU_img_name!=DSP_img_name:continue
            
        CPU_img=cv2.imread(CPU_path)
        DSP_img=cv2.imread(DSP_path)
        fing,ax=plt.subplots(1,2,figsize=(10,5))
        ax[0].imshow(CPU_img)
        ax[0].set_title("CPU 32 Inference")

        ax[1].imshow(DSP_img)
        ax[1].set_title("DSP 8 Inference")
        plt.tight_layout()
        plt.show()


# Mixed precision

In [ ]:
## Give appropriate permission to the directory "FOLDER_WITH_ARTIFACTS" you are working with
import os
os.environ['SNPE_ROOT']="../../../../../../snpe/snpe-2.10.0.4541"#set up your snpe path here.
os.environ['RAW_FILE_FOLDER']="raw"#rawfiles
os.environ['FOLDER_WITH_ARTIFACTS']="SSD_MobileNetV2"
os.environ['DLC32']="dlc/ssd_mobilenetV2_without_ABP-NMS.dlc"
os.environ['DLC8']="dlc/ssd_mobilenetV2_without_ABP-NMS_Q.dlc"
os.environ['TARGET_INPUT_LIST']="list.txt"
os.environ['ONDEVICE_FOLDER']="SSD_MobileNetV2"
os.environ['DEVICE_HOST']="localhost"
os.environ['DEVICE_ID']="e4149b7f" #fill your device-id. Use command "adb devices" to get devices names. example :"e18d5d0"
os.environ['SNPE_TARGET_ARCH']="aarch64-android-clang8.0"
os.environ['SNPE_TARGET_STL']="libc++_shared.so"

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell "mkdir -p /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/bin" && $DEVICE_SHELL shell "mkdir -p /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/lib" && $DEVICE_SHELL shell "mkdir -p /data/local/tmp/snpeexample/dsp/lib"

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL push $SNPE_ROOT/lib/$SNPE_TARGET_ARCH/$SNPE_TARGET_STL /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/lib
$DEVICE_SHELL push $SNPE_ROOT/bin/$SNPE_TARGET_ARCH/snpe-net-run /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/bin
$DEVICE_SHELL push $SNPE_ROOT/lib/dsp/*.so /data/local/tmp/snpeexample/dsp/lib
$DEVICE_SHELL push $SNPE_ROOT/lib/$SNPE_TARGET_ARCH/*.so /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/lib


In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell "mkdir -p /data/local/tmp/$ONDEVICE_FOLDER"

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL push $DLC32 /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push $DLC8 /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push raw /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push $TARGET_INPUT_LIST /data/local/tmp/$ONDEVICE_FOLDER

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/data/local/tmp/snpeexample/aarch64-android-clang8.0/lib
export PATH=$PATH:/data/local/tmp/snpeexample/aarch64-android-clang8.0/bin
export OUTPUT_FOLDER=OUTPUT_8b_DSP
export OUTPUT_DLC_QUANTIZED8=ssd_mobilenetV2_without_ABP-NMS_Q.dlc
export ADSP_LIBRARY_PATH="/data/local/tmp/snpeexample/dsp/lib;/system/lib/rfsa/adsp;/system/vendor/lib/rfsa/adsp;/dsp"
export ONDEVICE_FOLDER="SSD_MobileNetV2"
cd /data/local/tmp/$ONDEVICE_FOLDER &&
snpe-net-run --container $OUTPUT_DLC_QUANTIZED8 --input_list list.txt --output_dir $OUTPUT_FOLDER --use_dsp --debug

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/data/local/tmp/snpeexample/aarch64-android-clang8.0/lib
export PATH=$PATH:/data/local/tmp/snpeexample/aarch64-android-clang8.0/bin
export OUTPUT_FOLDER=OUTPUT_32b_CPU
export OUTPUT_DLC_32=ssd_mobilenetV2_without_ABP-NMS.dlc
export ONDEVICE_FOLDER="SSD_MobileNetV2"
cd /data/local/tmp/$ONDEVICE_FOLDER &&
snpe-net-run --container $OUTPUT_DLC_32 --input_list list.txt --output_dir $OUTPUT_FOLDER --debug

In [ ]:
%%bash
rm -rf OUTPUT_32b_CPU
rm -rf OUTPUT_8b_DSP
ls

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL pull /data/local/tmp/$ONDEVICE_FOLDER/OUTPUT_8b_DSP OUTPUT_8b_DSP
$DEVICE_SHELL pull /data/local/tmp/$ONDEVICE_FOLDER/OUTPUT_32b_CPU OUTPUT_32b_CPU

In [ ]:
%%bash
snpe-dlc-info -i dlc/ssd_mobilenetV2_without_ABP-NMS_Q.dlc >ssd_mobilenetV2_without_ABP-NMS_Q.txt
snpe-dlc-viewer -i dlc/ssd_mobilenetV2_without_ABP-NMS_Q.dlc -s ssd_mobilenetV2_without_ABP-NMS_Q.html

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
def cosine_sim(out_a, out_b):
    """
    This function computes the cosine similarity of two outputs.
    :param out_a: onnx output tensor
    :param out_b: system output tensor
    :return: cosine similarity
    """
    out_a = out_a.reshape(1,len(out_a))
    out_b = out_b.reshape(1,len(out_b))
    return cosine_similarity(out_a,out_b)[0][0]


def plot_layer_by_layer(INT8_DSP, FP32_CPU, output_folder):
    list_htp_nodes = [os.path.join(path, name) for path, subdirs, files in os.walk(INT8_DSP) for name in files if name.endswith("raw")]
    list_htp_nodes_mse = []
    final_node_names = []
    list_htp_nodes_no_raw = [ k.split('/')[-1].split('.')[0] for k in list_htp_nodes]
    for i in range(len(list_htp_nodes)):
        node_name = str(list_htp_nodes[i])
        htp_node_data =  np.fromfile(node_name,dtype=np.float32)
        if os.path.exists(node_name.replace(INT8_DSP.split("/")[1],FP32_CPU.split("/")[1])):
            cpu_node_data =  np.fromfile(node_name.replace(INT8_DSP.split("/")[1],FP32_CPU.split("/")[1]),dtype=np.float32)
        else:
            layer_new_name = node_name.replace(INT8_DSP.split("/")[1],FP32_CPU.split("/")[1])
            layer_new_name = layer_new_name.replace("_converted_UFIXED_POINT_16","")
            cpu_node_data =  np.fromfile(layer_new_name,dtype=np.float32)
            print("new_path :: ",layer_new_name)
#         cpu_node_data =  np.fromfile(node_name.replace("OUTPUT_8b_DSP","OUTPUT_FP32_CPU"),dtype=np.float32)
        error = cosine_sim(htp_node_data,cpu_node_data)
        list_htp_nodes_mse.append(error) # saving cosine similarity scores
        final_node_names.append(node_name.split("Result_0")[-1]) # saving layer names
    plt.figure(figsize=(30, 32))
    plt.plot(final_node_names,list_htp_nodes_mse)
    plt.savefig(f"{output_folder}/cosine_plot.png")
    plotting.output_file(f"{output_folder}/cosine_plot.html")
    plot = plotting.figure(x_range=final_node_names, height=200, title=f"rmse_plot_htp_vs_cpu", x_axis_label="Layers", y_axis_label="cosine")    
    plot.x(final_node_names,list_htp_nodes_mse)
    plot.line(final_node_names,list_htp_nodes_mse)
    plot.xaxis.major_label_orientation = "vertical"
    plot.sizing_mode = "scale_width"
    plotting.save(plot)
    

In [ ]:
output_folder = 'plots_fp32_vs_int8'
os.makedirs(output_folder,exist_ok=True)
INT8_DSP = "./OUTPUT_8b_DSP/Result_0"  # int8 dsp layer dump dir path
FP32_CPU = "./OUTPUT_32b_CPU/Result_0" # fp32 arm apu dump dir path
plot_layer_by_layer(INT8_DSP, FP32_CPU, output_folder)

In [ ]:
def get_scale_offset(min_value,max_value,bw=8):
    qmin = 0
    qmax = (2**bw)-1
    scale = (max_value - min_value) / (qmax - qmin)
    initial_zero_point = qmin - min_value / scale
    final_zero_point = 0;
    if (initial_zero_point < qmin):
        final_zero_point = qmin
    elif (initial_zero_point > qmax):
        final_zero_point = qmax
    else:
        final_zero_point = round(initial_zero_point)
    return scale,-final_zero_point

In [ ]:
# output-685 (1131)
scale, offset = get_scale_offset(0.0,0.984115421772, 16)
print("Sigmoid layer : scale:",scale," || offset:",offset )

In [ ]:
#Output-647(1104)
scale, offset = get_scale_offset(0.0,1.380036115646, 16)
print("Sigmoid layer : scale:",scale," || offset:",offset )

In [ ]:
%%bash
snpe-onnx-to-dlc -i onnx-models/ssd_mobilenetV2_without_ABP-NMS.onnx -o dlc/ssd_mobilenetV2_without_ABP-NMS.dlc --quantization_overrides encoding_format.encodings 


In [ ]:
%%bash
snpe-dlc-quantize --input_dlc dlc/ssd_mobilenetV2_without_ABP-NMS.dlc --override_params --output_dlc dlc/ssd_mobilenetV2_without_ABP-NMS_Q.dlc --input_list list.txt --enable_htp


In [ ]:
import os
os.environ['SNPE_ROOT']="../../../../../../snpe/snpe-2.12.0.230626"#set up your snpe path here.
os.environ['RAW_FILE_FOLDER']="raw"#rawfiles
os.environ['FOLDER_WITH_ARTIFACTS']="SSD_MobileNetV2"
os.environ['DLC32']="dlc/ssd_mobilenetV2_without_ABP-NMS.dlc"
os.environ['DLC8']="dlc/ssd_mobilenetV2_without_ABP-NMS_Q.dlc"
os.environ['TARGET_INPUT_LIST']="list.txt"
os.environ['ONDEVICE_FOLDER']="SSD_MobileNetV2"
os.environ['DEVICE_HOST']="localhost"
os.environ['DEVICE_ID']="e4149b7f" #fill your device-id. Use command "adb devices" to get devices names. example :"e18d5d0"
os.environ['SNPE_TARGET_ARCH']="aarch64-android"
os.environ['SNPE_TARGET_STL']="libc++_shared.so"

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell "mkdir -p /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/bin" && $DEVICE_SHELL shell "mkdir -p /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/lib" && $DEVICE_SHELL shell "mkdir -p /data/local/tmp/snpeexample/dsp/lib"

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL push $SNPE_ROOT/lib/$SNPE_TARGET_ARCH/$SNPE_TARGET_STL /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/lib
$DEVICE_SHELL push $SNPE_ROOT/bin/$SNPE_TARGET_ARCH/snpe-net-run /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/bin
$DEVICE_SHELL push $SNPE_ROOT/lib/hexagon-v73/unsigned/*.so /data/local/tmp/snpeexample/dsp/lib
$DEVICE_SHELL push $SNPE_ROOT/lib/$SNPE_TARGET_ARCH/*.so /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/lib

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell "mkdir -p /data/local/tmp/$ONDEVICE_FOLDER"

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL push $DLC32 /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push $DLC8 /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push raw /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push $TARGET_INPUT_LIST /data/local/tmp/$ONDEVICE_FOLDER

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/data/local/tmp/snpeexample/aarch64-android/lib
export PATH=$PATH:/data/local/tmp/snpeexample/aarch64-android/bin
export OUTPUT_FOLDER=OUTPUT_8b_DSP
export OUTPUT_DLC_QUANTIZED8=ssd_mobilenetV2_without_ABP-NMS_Q.dlc
export ADSP_LIBRARY_PATH="/data/local/tmp/snpeexample/dsp/lib;/system/lib/rfsa/adsp;/system/vendor/lib/rfsa/adsp;/dsp"
export ONDEVICE_FOLDER="SSD_MobileNetV2"
cd /data/local/tmp/$ONDEVICE_FOLDER &&
snpe-net-run --container $OUTPUT_DLC_QUANTIZED8 --input_list list.txt --set_output_tensors 935,986 --output_dir $OUTPUT_FOLDER --use_dsp

In [ ]:
%%bash

export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell

export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/data/local/tmp/snpeexample/aarch64-android/lib
export PATH=$PATH:/data/local/tmp/snpeexample/aarch64-android/bin
export OUTPUT_FOLDER=OUTPUT_32b_CPU
export OUTPUT_DLC_32=ssd_mobilenetV2_without_ABP-NMS.dlc
export ONDEVICE_FOLDER="SSD_MobileNetV2"
cd /data/local/tmp/$ONDEVICE_FOLDER &&
snpe-net-run --container $OUTPUT_DLC_32 --input_list list.txt --set_unconsumed_as_output --output_dir $OUTPUT_FOLDER

In [ ]:
%%bash
rm -rf OUTPUT_32b_CPU
rm -rf OUTPUT_8b_DSP
ls

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL pull /data/local/tmp/$ONDEVICE_FOLDER/OUTPUT_8b_DSP OUTPUT_8b_DSP
$DEVICE_SHELL pull /data/local/tmp/$ONDEVICE_FOLDER/OUTPUT_32b_CPU OUTPUT_32b_CPU

In [ ]:
#000000329219
folder = ["OUTPUT_32b_CPU","OUTPUT_8b_DSP"]
detection_results = ["detection_results_32b_CPU","detection_results_8b_DSP"]
results = ["prediction_results_32b_CPU","prediction_results_8b_DSP"]
f=open("list.txt","r")
r=0.31
input_shape = tuple(map(int,[320,320]))
input_list = f.readlines()
f.close()
for j in range(0,2):
    for result_path in glob.glob(os.path.join(folder[j], '*')):
        if ".log" not in result_path and int(result_path.split("_")[-1])<len(input_list):
            #using raw file to traverse output folders and result folder will be in sequence as per list.txt
            raw_path = input_list[int(result_path.split("_")[-1])].replace("\n","") # rawfile path from list.txt
            if os.path.exists(raw_path):
                scores = np.fromfile(result_path+'/935.raw', dtype="float32")
                boxes=np.fromfile(result_path+'/986.raw', dtype="float32")

                boxes=boxes.reshape((3234,4))
                scores=scores.reshape((3234,21))
                originalImagePath = raw_path.replace(".raw",".jpg").replace("raw","val2017")

                runtime = folder[j].split("_")[-1]
                save_img_path = detection_results[j]+"/"+raw_path.split("/")[-1].split(".")[0]+"_"+runtime+".jpg"

                postProcessing(scores,boxes,originalImagePath,save_img_path)
                

In [ ]:
detection_results = ["detection_results_32b_CPU","detection_results_8b_DSP"]

for CPU_path in glob.glob(os.path.join(detection_results[0], '*')):
    CPU_img_name=CPU_path.split("/")[1].split("_")[0]
    for DSP_path in glob.glob(os.path.join(detection_results[1], '*')):
        DSP_img_name=DSP_path.split("/")[1].split("_")[0]
        
        if CPU_img_name!=DSP_img_name:continue
            
        CPU_img=cv2.imread(CPU_path)
        DSP_img=cv2.imread(DSP_path)
        fing,ax=plt.subplots(1,2,figsize=(10,5))
        ax[0].imshow(CPU_img)
        ax[0].set_title("CPU 32 Inference")

        ax[1].imshow(DSP_img)
        ax[1].set_title("DSP 8 Inference")
        plt.tight_layout()
        plt.show()
